In [41]:
#import  package
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np
import datetime as dt

#import  data
train = pd.read_csv("E:/WORKOUT/Statistic/data_sciense_intro/report1/train.csv")
test = pd.read_csv("E:/WORKOUT/Statistic/data_sciense_intro/report1/test.csv")

#資料前處理
# 將次數過少的action_type先去除
X_action_type = train['action_type'].value_counts()
start = X_action_type.index.get_loc('Running Jump Shot')
all_action_name = list(X_action_type.index)[start:57]
train = train[~train.action_type.isin(all_action_name)]

#將為類別變數型態資料轉換為 str 型態
#對loc_x loc_y取絕對值
X_train = train.drop(columns=['action_type','shot_id'])
X_test = test.drop(columns=['shot_id'])
Y_train= np.array(train["action_type"])
X_train['game_date']=pd.to_datetime(X_train['game_date'])
epoch = dt.datetime(1970, 1, 1)
step = 0
temp =[]
for t in [(d - epoch).total_seconds() for d in X_train['game_date']]:
    temp.append('%.0f' % t)  
X_train.drop('game_date', axis = 1, inplace = True)
X_train['game_date'] = temp
X_train['game_date'] = X_train['game_date'].astype('float32')
mean_game_date= X_train['game_date'].mean()
std_game_date= X_train['game_date'].std()
X_train['game_date'] = (X_train['game_date'] - mean_game_date) / (std_game_date)

X_test['game_date']=pd.to_datetime(X_test['game_date'])
epoch = dt.datetime(1970, 1, 1)
step = 0
temp =[]
for t in [(d - epoch).total_seconds() for d in X_test['game_date']]:
    temp.append('%.0f' % t)  
X_test.drop('game_date', axis = 1, inplace = True)
X_test['game_date'] = temp
X_test['game_date'] = X_test['game_date'].astype('float32')
X_test['game_date'] = (X_test['game_date'] - mean_game_date) / (std_game_date)

data=X_train.append(X_test)
data["playoffs"]=data["playoffs"].astype("category").cat.codes
data["period"]=data["period"].astype("category").cat.codes
data["season"]=data["season"].astype("category").cat.codes
data["shot_made_flag"]=data["shot_made_flag"].astype("category").cat.codes
data["shot_zone_area"]=data["shot_zone_area"].astype("category").cat.codes
data["opponent"]=data["opponent"].astype("category").cat.codes

X_train=data[0:X_train.shape[0]]
X_test=data[X_train.shape[0]:data.shape[0]]

In [42]:
#Random Forest model
rf = RandomForestClassifier(n_estimators=500,oob_score=True,min_samples_split=50)
rf.fit(X_train, Y_train)

from sklearn.metrics import accuracy_score
predicted = rf.predict(X_train)
accuracy = accuracy_score(Y_train, predicted)
print(rf.oob_score_)
print(accuracy)

0.853930933137399
0.8681361743815822


In [44]:
#model predict
final_pred=rf.predict(X_test)
#create output
output=pd.DataFrame()
output[0]=list(test['shot_id'])
output[1]=final_pred
output.to_csv('E:/WORKOUT/Statistic/data_sciense_intro/report1/rf_out.csv', index = False, header = False)